In [1]:
import json
import pandas as pd

# Constants

In [2]:
# CONSTANTS

# Object template to communicate data to the CM
TEMPLATE = {
    "id": None,
    "userid": None,
    "origin": None,
    "source_id": "fake90e6d701748f08514b01",
    "source": "fake",
    "pname": None,
    "pvalue": None,
    "context": "application",
    "datapoints": 0,
    "category": None
}

FOLDER = "./data/"

_ID = 0


# Users

It transforms demographic data from a LDH dataset into the ugc data sent to the CM by the UM

In [17]:
# Constants used when transforming demographic data
C_USER =  dict(
    datafile ="GAM_122-Gam_Game_Users.json",
    demographicAtts = ['Gender', 'Age', 'RelationshipWithArt','RelationshipWithMuseum', 'ContentInLIS'],
    userIdAtt = 'authorUsername',
    ugcfile = "ugcUsers.json"
)

In [18]:
with open(FOLDER+C_USER["datafile"]) as f:
    userData = json.load(f)


In [19]:
postDict= {}
for user in userData:    
    result = []
    currentData = TEMPLATE.copy()
    currentData["category"] = "demographics"
    currentData["userid"] = user[C_USER["userIdAtt"]]
    for item in C_USER["demographicAtts"]:
        attData = currentData.copy()
        attData["id"] = _ID
        _ID+=1
        attData["origin"] = "User"
        attData["pname"]  = item
        attData["pvalue"] = user[item] 
        result.append(attData)
    if user[C_USER["userIdAtt"]] in postDict:
        print("Duplicated user", user[C_USER["userIdAtt"]])
    postDict[user[C_USER["userIdAtt"]]] = result

In [20]:
with open(FOLDER+C_USER["ugcfile"], "w") as outfile:
    json.dump(postDict, outfile, indent=2)

In [22]:
# postDict is a dictionary with userids as fields and arrays of demographic data for the CM as values

In [23]:
# Execute POST:
# server = "http://localhost:8080"
# for key, value in postDict.items():
#    response=requests.post(f'{server}/v1.1/users/{key}/update-generated-content', json = value)
#    procesar el resultado de response

# User contributions



In [3]:
# Constants related with user contributions
C_CONTRIB = dict(
    datafile = "GAMStories_29Dec2022.json",
    userIdAtt = 'authorUsername',
    artifactIdAtt = 'artworkId',
    contribIdAtt = '_id',
    contribAtts = ['itMakesMeFeel','itMakesMeThinkAbout','itRemindsMeOf'],
    contribAttValues = ['emotions', 'sentiment'],
    ugcfile = "ugcContributions.json"
)

In [4]:
with open(FOLDER+C_CONTRIB["datafile"]) as f:
    contribData = json.load(f)
contribData

[{'_id': '6364f16d7b76095db541cb9f',
  'title': 'Butterfly effect',
  'parts': [{'artworkId': '35360',
    'multimediaData': {'answersToTemplates': {'itMakesMeFeel': '',
      'itMakesMeThinkAbout': 'Amour',
      'itRemindsMeOf': ''},
     'emojis': [{'locationX': 0.375, 'locationY': 0.3, 'emoji': '🥰'}],
     'tags': [],
     'analysis': {'itMakesMeFeel': {},
      'itMakesMeThinkAbout': {'emotions': {},
       'sentiment': {'Positive': 0.84607714},
       'toxicity': {},
       'entities': {}},
      'itRemindsMeOf': {}}}},
   {'artworkId': '35456',
    'multimediaData': {'answersToTemplates': {'itMakesMeFeel': '',
      'itMakesMeThinkAbout': 'Ritrovarsi',
      'itRemindsMeOf': ''},
     'emojis': [{'locationX': -0.018003216911764704,
       'locationY': 0.069091796875,
       'emoji': '😌'}],
     'tags': [{'locationX': 0.375, 'locationY': 0.3, 'tag': 'Peace'}],
     'analysis': {'itMakesMeFeel': {},
      'itMakesMeThinkAbout': {'emotions': {},
       'sentiment': {'Negative': 0.8

In [8]:
def createUserContribution(contrib, contribAtt, contribValue):
    currentData = TEMPLATE.copy()
    currentData["category"] = "interest"
    currentData["userid"] = contrib[C_CONTRIB["userIdAtt"]]
    currentData["id"] = contrib[C_CONTRIB["contribIdAtt"]]
    interactions = []
    # OJO: El separador "." no le gusta a Python en los dict. Yo lo cambiaría por "_"
    sep = "."
    for interaction in contrib["parts"]:
        interactionData = currentData.copy()
        interactionData["origin"] = interaction[C_CONTRIB["artifactIdAtt"]]
        interactionData["pname"] = f'{contribAtt}{sep}{contribValue}'
        analysysValue =  interaction["multimediaData"]["analysis"][contribAtt]
        pvalue = analysysValue[contribValue] if contribValue in analysysValue else ""
        pvalue = "" if pvalue == [] else pvalue
        interactionData["pvalue"] = pvalue
        interactions.append(interactionData)
    return interactions

In [9]:
def createUserTextContribution(contrib, contribAtt):
    currentData = TEMPLATE.copy()
    currentData["category"] = "interest"
    currentData["userid"] = contrib[C_CONTRIB["userIdAtt"]]
    currentData["id"] = contrib[C_CONTRIB["contribIdAtt"]]
    interactions = []
    # OJO: El separador "." no le gusta a Python en los dict. Yo lo cambiaría por "_"
    sep = "."
    for interaction in contrib["parts"]:
        interactionData = currentData.copy()
        interactionData["origin"] = interaction[C_CONTRIB["artifactIdAtt"]]
        # Add text (e.g. itMakesMeFeel.text)
        interactionData["pname"] = f'{contribAtt}{sep}text'
        textValue =  interaction["multimediaData"]["answersToTemplates"][contribAtt]
        # textValue = textValue.encode("ascii", "ignore").decode()
        interactionData["pvalue"] = textValue
        interactions.append(interactionData)
    return interactions


In [16]:
def createUserEmojiContribution(contrib):
    currentData = TEMPLATE.copy()
    currentData["category"] = "interest"
    currentData["userid"] = contrib[C_CONTRIB["userIdAtt"]]
    currentData["id"] = contrib[C_CONTRIB["contribIdAtt"]]
    interactions = []
    # OJO: El separador "." no le gusta a Python en los dict. Yo lo cambiaría por "_"
    sep = "."
    for interaction in contrib["parts"]:
        interactionData = currentData.copy()
        interactionData["origin"] = interaction[C_CONTRIB["artifactIdAtt"]]
        interactionData["pname"] = f'emotions{sep}emojis'
        emojis =  interaction["multimediaData"]["emojis"]
        emojiValue = [emoji['emoji'] for emoji in emojis]
        emojiValue = [*set(emojiValue)] # Remove duplicates
        pvalue = "" if emojiValue == [] else emojiValue
        interactionData["pvalue"] = pvalue
        interactions.append(interactionData)
    return interactions

In [17]:
postDict= {}

for contribAtt in C_CONTRIB["contribAtts"]:
    for contribValue in C_CONTRIB["contribAttValues"]:
        key = f'{contribAtt}_{contribValue}'
        attValueDict = {}
        for contrib in contribData: 
            result = createUserContribution(contrib, contribAtt, contribValue)
            attValueDict[contrib[C_CONTRIB["userIdAtt"]]] = result
        postDict[key] = attValueDict

        # The text is different because it is not in analysis but in answersToTemplates
        key = f'{contribAtt}_text'
        attValueDict = {}
        for contrib in contribData: 
            result = createUserTextContribution(contrib, contribAtt)
            attValueDict[contrib[C_CONTRIB["userIdAtt"]]] = result
        postDict[key] = attValueDict
        
for contrib in contribData: 
    key = "emojis"
    result = createUserEmojiContribution(contrib)
    attValueDict[contrib[C_CONTRIB["userIdAtt"]]] = result
    postDict[key] = attValueDict
        
postDict

{'itMakesMeFeel_emotions': {'HoNIcWZY': [{'id': '6364f4e9ad16cc2ce70b6f0f',
    'userid': 'HoNIcWZY',
    'origin': '35330',
    'source_id': 'fake90e6d701748f08514b01',
    'source': 'fake',
    'pname': 'itMakesMeFeel.emotions',
    'pvalue': '',
    'context': 'application',
    'datapoints': 0,
    'category': 'interest'},
   {'id': '6364f4e9ad16cc2ce70b6f0f',
    'userid': 'HoNIcWZY',
    'origin': '35229',
    'source_id': 'fake90e6d701748f08514b01',
    'source': 'fake',
    'pname': 'itMakesMeFeel.emotions',
    'pvalue': '',
    'context': 'application',
    'datapoints': 0,
    'category': 'interest'},
   {'id': '6364f4e9ad16cc2ce70b6f0f',
    'userid': 'HoNIcWZY',
    'origin': '35380',
    'source_id': 'fake90e6d701748f08514b01',
    'source': 'fake',
    'pname': 'itMakesMeFeel.emotions',
    'pvalue': '',
    'context': 'application',
    'datapoints': 0,
    'category': 'interest'}],
  'e4aM9WL7': [{'id': '622764d6efc0335e1948a813',
    'userid': 'e4aM9WL7',
    'origin'

In [61]:
# Example for a key in postDict
postDict["itRemindsMeOf_emotions"]

{'HoNIcWZY': [{'id': '6364f4e9ad16cc2ce70b6f0f',
   'userid': 'HoNIcWZY',
   'origin': '35330',
   'source_id': 'fake90e6d701748f08514b01',
   'source': 'fake',
   'pname': 'itRemindsMeOf.emotions',
   'pvalue': '',
   'context': 'application',
   'datapoints': 0,
   'category': 'interest'},
  {'id': '6364f4e9ad16cc2ce70b6f0f',
   'userid': 'HoNIcWZY',
   'origin': '35229',
   'source_id': 'fake90e6d701748f08514b01',
   'source': 'fake',
   'pname': 'itRemindsMeOf.emotions',
   'pvalue': '',
   'context': 'application',
   'datapoints': 0,
   'category': 'interest'},
  {'id': '6364f4e9ad16cc2ce70b6f0f',
   'userid': 'HoNIcWZY',
   'origin': '35380',
   'source_id': 'fake90e6d701748f08514b01',
   'source': 'fake',
   'pname': 'itRemindsMeOf.emotions',
   'pvalue': '',
   'context': 'application',
   'datapoints': 0,
   'category': 'interest'}],
 'e4aM9WL7': [{'id': '622764d6efc0335e1948a813',
   'userid': 'e4aM9WL7',
   'origin': '35229',
   'source_id': 'fake90e6d701748f08514b01',
   '

In [18]:
with open(FOLDER+C_CONTRIB["ugcfile"], "w") as outfile:
    json.dump(postDict, outfile, indent=2)

In [63]:
# postDict is a dictionary where:
# - Keys are the interaction/contribution types that we are interested in to send to CM
# For example: itRemindsMeOf_text key contains all the contributions with the text of "itRemindsMeOf" (whose pname is itRemindsMeOf.text)
# - Values are dictionaries where keys are userIds and values are a list of contributions for this user of the type described by the key.

# # To POST users to a CM server in localhost:8080 uncomment next lines
# server = "http://localhost:8080"
# contribTypes = ["itRemindsMeOf_text", "itRemindsMeOf_emotions"] 
# for ctype in contribTypes:
#    contribsDict = postDict[cType]
#    for key, value in contribsDict.items():
#        response=requests.post(f'{server}/v1.1/users/{key}/update-generated-content', json = value)
         # Process response value

In [19]:
postDict

{'itMakesMeFeel_emotions': {'HoNIcWZY': [{'id': '6364f4e9ad16cc2ce70b6f0f',
    'userid': 'HoNIcWZY',
    'origin': '35330',
    'source_id': 'fake90e6d701748f08514b01',
    'source': 'fake',
    'pname': 'itMakesMeFeel.emotions',
    'pvalue': '',
    'context': 'application',
    'datapoints': 0,
    'category': 'interest'},
   {'id': '6364f4e9ad16cc2ce70b6f0f',
    'userid': 'HoNIcWZY',
    'origin': '35229',
    'source_id': 'fake90e6d701748f08514b01',
    'source': 'fake',
    'pname': 'itMakesMeFeel.emotions',
    'pvalue': '',
    'context': 'application',
    'datapoints': 0,
    'category': 'interest'},
   {'id': '6364f4e9ad16cc2ce70b6f0f',
    'userid': 'HoNIcWZY',
    'origin': '35380',
    'source_id': 'fake90e6d701748f08514b01',
    'source': 'fake',
    'pname': 'itMakesMeFeel.emotions',
    'pvalue': '',
    'context': 'application',
    'datapoints': 0,
    'category': 'interest'}],
  'e4aM9WL7': [{'id': '622764d6efc0335e1948a813',
    'userid': 'e4aM9WL7',
    'origin'